# Importing the libraries

In [1]:
from pandasdmx import Request

In [2]:
import pandas as pd

In [3]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

In [4]:
output_notebook()

Loading BokehJS ...

# Connecting to the webservice

In [ ]:
estat = Request('ESTAT')

### Downloading dataflow definitions

In [ ]:
dflow = estat.dataflow()

## Defining functions

### Data structure

In [ ]:
def dsd_resp(id):
    """
    The function takes a dataflow identifier and requests from the Eurostat SDMX api the dataflow's datastructure, datastructure id and datastructure response.
    """    
    # Defining the data set's dataflow based on the data set id.
    df = dflow.dataflow[id]
    
    # Printing the table's name
    print("The table's name:")
    print(dflow.write().dataflow.loc[id][0])
    
    # Acquiring the dataflow's datastructure id
    dsd_id = df.structure.id
    
    # Requesting for the dataflow's datastructure
    dsd_resp = estat.get(resource_type = 'datastructure', resource_id = dsd_id)
    
    # Requesting the dataflow's datastructure
    dsd = dsd_resp.datastructure[dsd_id]
    
    return dsd_resp, dsd_id, dsd

### Datastructure attributes

In [ ]:
def dsd_att(dsd):
    """The function takes a table datastructure definition and prints its measure, dimension and attribute lists."""
    # Measures
    print("Measures:")
    print(dsd.measures.aslist())
    
    # Dimensions
    print("\nDimensions:")
    print(dsd.dimensions.aslist())
    
    # Attributes
    print("\nAttributes:")
    print(dsd.attributes.aslist())

## Selecting the tables

### Listing the tables from the 'High-tech industry and knowledge-intensive services (**htec**)' database

In [ ]:
dflow.write().dataflow[dflow.write().dataflow.index.str.startswith('htec') == True]

### The tables I am going to analyze

In [ ]:
id1 = 'htec_sti_exp2'

In [ ]:
dflow.write().dataflow.loc[id1][0]

In [ ]:
id2 = 'htec_vci_stage2'

In [ ]:
dflow.write().dataflow.loc[id2][0]

# Venture capital data

### Getting the table's dataflow definition and datastructure

In [ ]:
dsd_resp2, dsd_id2, dsd2 = dsd_resp(id2)

### Downloading the table's datastructure definition

In [ ]:
dsd_att(dsd2)

Codelist:

In [ ]:
dsd_resp2.write().codelist.loc[:,:] # [['EXPEND', 'UNIT'],:]

### Requesting the data

In [ ]:
dresp2 = estat.get(resource_type = 'data',
                   resource_id = id2,
                   params = {'references': None,
                             #'startPeriod': '2014',
                             #'endPeriod': '2014'
                            })

In [ ]:
dat2 = dresp2.data

In [ ]:
dat2.dim_at_obs

In [ ]:
ser2 = list(dat2.series)
len(ser2)

In [ ]:
ser2[5].key

In [ ]:
set(s.key.UNIT for s in dat2.series)

### Writing the data into DataFrame

In [ ]:
iter2 = (s for s in dat2.series if (s.key.GEO in ['EU15']) == False)
tab2 = dresp2.write(iter2)
tab2 = tab2.stack('GEO')
tab2.columns = tab2.columns.droplevel(['FREQ']) # Dropping 'FREQ' and 'UNIT' levels
tab2.swaplevel('UNIT', 'EXPEND', axis = 1)
tab2 = tab2.loc['2015', ['PC_GDP', 'MIO_EUR']]
# tab2.dropna(axis = 0, how = 'any', inplace = True)
tab2.index = tab2.index.droplevel(0) # Dropping the 'TIME_PERIOD' axis
tab2

In [ ]:
source = ColumnDataSource(data =
                         dict(names = list(tab2.index),
                              invest = tab2.loc[:,('PC_GDP','INV')],
                              venture = tab2.loc[:,('PC_GDP','INV_VEN')],
                              mileur = list(tab2.loc[:,('MIO_EUR','INV')] / 500))
                         )


p = figure(plot_width = 600, plot_height = 600)
p.square(x = tab2.loc[:,('PC_GDP','INV')],
         y = tab2.loc[:,('PC_GDP','INV_VEN')],
         size = mileur,
         color = "blue",
         alpha = 0.6)

p.xaxis.axis_label = 'Total R&D Investment'
p.yaxis.axis_label = 'Venture Capital R&D Investment'

labels = LabelSet(x = 'invest',
                  y = 'venture',
                  text = 'names',
                  source = source,
                  render_mode='canvas',
                  level='glyph')

p.add_layout(labels)
show(p)